In [1]:
import duckdb
from IPython.display import display, HTML
import ipywidgets as widgets
import polars as pl
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
conn = duckdb.connect("/home/jovyan/datasets/open_payments/duckdb/openpayments.duckdb", read_only=True)
tables = conn.sql("show tables").to_df()['name'].values
tables.sort()
tableswidget=widgets.Dropdown(
    options=tables,
    value=tables[0],
    description='Dataset:',
    disabled=False,
)
display(HTML("<h1>Openpayments Dataset Extractor</h1>"))
@widgets.interact(tables=tableswidget)
def on_select_table(table=tableswidget):
    if table != 'covered_recipients':
        years = list(conn.sql(f"select distinct Program_Year from {table} order by Program_Year desc").to_df()['Program_Year'].values)
        years.insert(0,"ALL")
        yearwidget = tableswidget=widgets.Dropdown(options=years,value=years[1], description='Choose Year:', disabled=False)
        @widgets.interact(year=yearwidget)
        def on_select_year(year):
            if year == "ALL":
                rows = conn.sql(f"select count(*) as row_count from {table}").to_df()['row_count'].values[0]
            else:
                rows = conn.sql(f"select count(*) as row_count from {table} where Program_Year={year}").to_df()['row_count'].values[0]

            rowswidget = widgets.IntSlider( value=rows//2,min=0,max=rows,step=250, description='Rnd Sample:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d'
                )
            display(HTML(f"Rows in Dataset: {rows:,}"))
            @widgets.interact_manual(sample=rowswidget)
            def on_click(sample):
                display(HTML(f"<p>dataset:<code>{table}</code> year: <code>{year}</code>, random sample:<code>{sample:,}</code> of <code>{rows:,}</code>"))
                filename = f"{table}-{year}-sample-{sample}.csv".lower()
                display(HTML(f"<b>Saving File:</b> <code>{filename}</code>"))
                if year == "ALL":
                    dataset = conn.sql(f"select * from (select * from {table}) using sample {sample} rows")
                else:
                    dataset = conn.sql(f"select * from (select * from {table} where Program_Year={year}) using sample {sample} rows")
                dataset.write_csv(filename,header=True)
                display(HTML(f"<b>File Saved!</b>")) 

interactive(children=(Dropdown(description='Dataset:', options=('covered_recipients', 'deleted_records_details…